# Homework 5

#### Author: Gustavo Soares

##### Imports

In [1]:
import pandas as pd
import numpy as np
from dataapi import DBConnect, SGS
import matplotlib.pyplot as plt
from calendars import DayCounts
from tqdm import tqdm
%matplotlib inline

### Dados do DI Futuro
Servidor AWS do FinanceHub.

In [2]:
dbc = DBConnect(username='fhreadonly', password='finquant')
query = "select * from \"B3futures\" where contract='DI1'"
df = pd.read_sql(query, dbc.connection)
df.tail(10)

,time_stamp,contract,maturity_code,open_interest_open,open_interest_close,number_of_trades,trading_volume,financial_volume,previous_settlement,indexed_settlement,opening_price,minimum_price,maximum_price,average_price,last_price,settlement_price,last_bid,last_offer
157906,2021-03-05,DI1,F26,179178,179178,2472,18380,1298827582,70494.36,71215.07,7.54,7.29,7.62,7.460,7.33,71209.76,7.31,7.35
157907,2021-03-05,DI1,N26,38470,38470,13,100,6782387,67473.47,68280.74,7.63,7.46,7.63,7.586,7.46,68275.65,7.45,0.00
157908,2021-03-05,DI1,F27,523667,523667,9487,105710,6845765035,64447.59,65343.22,7.84,7.59,7.92,7.754,7.64,65338.35,7.62,7.64
157909,2021-03-05,DI1,F28,39950,39950,196,1610,95477890,58973.90,59987.28,8.07,7.81,8.12,7.970,7.84,59982.81,0.00,0.00
157910,2021-03-05,DI1,F29,217513,217513,4136,40135,2178737915,53903.46,55021.08,8.25,7.96,8.32,8.145,8.00,55016.98,7.98,8.02
157911,2021-03-05,DI1,F30,8020,8020,0,0,0,49191.87,50401.07,0.00,0.00,0.00,0.000,0.00,50397.31,0.00,0.00
157912,2021-03-05,DI1,F31,100422,100422,1436,13750,622193161,44831.97,46121.17,8.51,8.22,8.58,8.430,8.27,46117.73,8.22,8.28
157913,2021-03-05,DI1,F33,8530,8530,0,0,0,37263.61,38553.51,0.00,0.00,0.00,0.000,0.00,38550.63,0.00,0.00
157914,2021-03-05,DI1,F35,0,0,0,0,0,31015.21,32269.68,0.00,0.00,0.00,0.000,0.00,32267.27,0.00,0.00
157915,2021-03-05,DI1,F36,0,0,0,0,0,28509.87,29747.03,0.00,0.00,0.00,0.000,0.00,29744.81,0.00,0.00


### Cálculo dos dias úteis até o vencimento de um contrato

In [ ]:
df['time_stamp'] = pd.to_datetime(df['time_stamp'])

mat_dict = {'01': 'F',
            '02': 'G',
            '03': 'H',
            '04': 'J',
            '05': 'K',
            '06': 'M',
            '07': 'N',
            '08': 'Q',
            '09': 'U',
            '10': 'V',
            '11': 'X',
            '12': 'Z'}

mat_dict = {v: k for k, v in mat_dict.items()}

mat_month = df['maturity_code'].str[0].map(mat_dict)

def convert_year(year2digits):
    
    if int(year2digits) >= 91:
        year4digits = '19' + year2digits
    else:
        year4digits = '20' + year2digits
    
    return year4digits

mat_year = df['maturity_code'].str[-2:].apply(convert_year)

dates = pd.to_datetime(mat_year + mat_month + '01')

dc = DayCounts('BUS/252', calendar='anbima')
dates = dc.busdateroll(dates, 'following')
dates

df['maturity_date'] = dates

df['DU'] = dc.days(df['time_stamp'], df['maturity_date'])
df = df[df['DU'] >= 0]

# Utilizando apenas PUs não nulos
df = df[df.settlement_price != 0]

# Apenas Open Interest > 1
df = df[df['trading_volume'] != 0]

df.tail(35)

### Série do CDI de um dia
API do [Sistema Gerenciador de Series](https://www3.bcb.gov.br/sgspub/localizarseries/localizarSeries.do?method=prepararTelaLocalizarSeries) do BCB.

In [ ]:
sgs = SGS()
df_cdi = sgs.fetch({12: 'CDI'})/100
df_cdi.tail(5)

# Estratégias no futuro de DI-1 

## Q1: Carry

Usando o que você aprendeu sobre o sinal de carry em [Koijen, Moskowitz, Pedersen, and Vrugt (2016)](https://ssrn.com/abstract=2298565) e [Brooks and Moskowitz (2017)](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=2956411), faça o cálculo do carry anualizado esperado de se segurar um contrato de futuros DI-1 por 3 meses. Calcule o valor do carry para todos os contratos e todos os dias do dataframe ```df``` acima. Calcule o carry do contrato "puro" e também do carry de 3 meses ajustado pelo DV01 do contrato.

## Q2: Carry trade

Usando o que você aprendeu sobre carry portfolios em [Koijen, Moskowitz, Pedersen, and Vrugt (2016)](https://ssrn.com/abstract=2298565) e [Brooks and Moskowitz (2017)](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=2956411), faça o backtest de uma estratégia DV01 neutra que escolhe contratos de acordo com seu carry para criar DV01 flattners ou steepners. A estratégia final deve ser DV01 neutra (ou seja, ajuste os holdings por DV01 nas datas de rebalanceamento das posições). Não transacionar contratos que tem zero open interest ou zero volume.

## Q3: Market timing

Usando o que você aprendeu sobre carry timing em [Koijen, Moskowitz, Pedersen, and Vrugt (2016)](https://ssrn.com/abstract=2298565), faça o backtest de uma estratégia direcional que fica "tomada" ou "dada" em DI-1 de acordo com o sinal de carry. Faça com que seus trades tenham sempre o mesmo DV01 como percentual do notional investido, i.e., todo trade tenha o mesmo duration. Compare o resultado com estar constatemente "dado" no mesmo duration. Não transacionar contratos que tem zero open interest ou zero volume.